In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
artists = []

In [8]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [10]:
SEED = ["https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn_(ca_s%C4%A9,_sinh_1981)", 
        "https://vi.wikipedia.org/wiki/Miu_L%C3%AA", 
        "https://vi.wikipedia.org/wiki/H%C3%B2a_Minzy",
        "https://vi.wikipedia.org/wiki/M%E1%BB%B9_Linh",
        "https://vi.wikipedia.org/wiki/Only_C",
        "https://vi.wikipedia.org/wiki/JustaTee",
        "https://vi.wikipedia.org/wiki/Ch%E1%BA%BF_Linh",
        "https://vi.wikipedia.org/wiki/%C4%90%C3%A0m_V%C4%A9nh_H%C6%B0ng",
        "https://vi.wikipedia.org/wiki/Mr._Siro"
        ]

In [11]:
def remove_characters(text):
    if isinstance(text, str):
        # Bỏ cụm [sửa|sửa mã nguồn]
        cleaned = text.replace("[sửa|sửa mã nguồn]", "")
        # Xoá ngoặc và dấu cách/dấu ngoặc kép ở đầu & cuối
        cleaned = re.sub(r'^[\s\(\)\[\]\'"]+|[\s\(\)\[\]\'"]+$', '', cleaned)
        return cleaned.strip()
    return text


In [12]:
def get_years(active_years):
    is_active = False
    if not active_years:
        return None, None

    if isinstance(active_years, str):
        active_years = [active_years]

    start_years = []
    end_years = []

    for period in active_years:
        if not period:
            continue
        p = period.strip()

        # Chuẩn hoá các loại dash thành hyphen thường
        p = re.sub(r'[–—−]', '-', p)

        # 🔹 Lấy tất cả năm và cả từ "nay"
        tokens = re.findall(r'\b(?:19|20)\d{2}\b|\b(?:nay|hiện tại|present|now)\b', p, re.IGNORECASE)

        # Nếu không có token nào, thử kiểm tra dạng đặc biệt "2015-"
        if not tokens:
            if re.search(r'\b(?:19|20)\d{2}\b\s*-\s*$', p):
                start = int(re.search(r'(?:19|20)\d{2}', p).group())
                start_years.append(start)
            continue

        # Xử lý token đầu tiên (năm bắt đầu)
        first = tokens[0]
        if re.match(r'(?:19|20)\d{2}', first):
            start_years.append(int(first))

        # Xử lý token cuối cùng (năm tan rã)
        last = tokens[-1]
        if re.match(r'(?:19|20)\d{2}', last):
            end_years.append(int(last))
        elif re.match(r'(nay|hiện tại|present|now)', last, re.IGNORECASE):
            is_active = True
            # nếu là 'nay' thì không có năm tan rã
            pass
        elif len(tokens) == 1:
            # chỉ có một năm, coi là hoạt động trong năm đó
            end_years.append(int(first))

    if not start_years:
        return None, None

    start = min(start_years)
    end = None if is_active else max(end_years)
    return start, end


In [13]:
def crawl_singer_info(urls):
    singers = []
    for url in urls:
        try:
            print(f"Crawling {url}...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            #Trỏ vào info box
            info_box = soup.find("table", {"class": "infobox"})
            if not info_box:
                continue
            info_rows = info_box.find_all("tr")
            singer_info = {}
            singer_info['name'] = soup.find("h1", {"id": "firstHeading"}).get_text(strip=True)
            for row in info_rows:
                header = row.find("th")
                data = row.find("td")
                if header and data:
                    key = header.get_text(strip=True)

                    # --- TRƯỜNG HỢP 1: Có <div class="hlist"> ---
                    hlist_div = data.find("div", {"class": "hlist"})
                    if hlist_div:
                        # Lấy nội dung trong các <li>
                        items = [li.get_text(strip=True) for li in hlist_div.find_all("li")]
                        singer_info[key] = items
                        continue  # bỏ qua các xử lý tiếp theo, vì đã có kết quả

                    ul_tag = data.find("ul")
                    if ul_tag:
                        items = [li.get_text(strip=True) for li in ul_tag.find_all("li")]
                        singer_info[key] = items
                        continue

                    # --- TRƯỜNG HỢP 2: Có <br> ---
                    if data.find("br"):
                        # split dựa theo thẻ <br>
                        parts = [text.strip() for text in data.stripped_strings]
                        singer_info[key] = [p for p in parts if p]  # loại bỏ chuỗi rỗng
                    else:
                        # Trường hợp bình thường
                        value = data.get_text(separator=' ', strip=True)
                        singer_info[key] = value
                    
                    singer_info['năm thành lập'], singer_info['năm tan rã'] = get_years(singer_info.get('Năm hoạt động'))
            # print(f"Singer Info from {url}: {singer_info}")
            # singer_info['albums'] = crawl_band_albums(soup)
            singers.append(singer_info)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
    return singers

In [14]:
singers = crawl_singer_info(SEED)
print(singers)


Crawling https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn_(ca_s%C4%A9,_sinh_1981)...
Crawling https://vi.wikipedia.org/wiki/Miu_L%C3%AA...
Crawling https://vi.wikipedia.org/wiki/H%C3%B2a_Minzy...
Crawling https://vi.wikipedia.org/wiki/M%E1%BB%B9_Linh...
Crawling https://vi.wikipedia.org/wiki/Only_C...
Crawling https://vi.wikipedia.org/wiki/JustaTee...
Crawling https://vi.wikipedia.org/wiki/Ch%E1%BA%BF_Linh...
Crawling https://vi.wikipedia.org/wiki/%C4%90%C3%A0m_V%C4%A9nh_H%C6%B0ng...
Crawling https://vi.wikipedia.org/wiki/Mr._Siro...
[{'name': 'Lệ Quyên (ca sĩ, sinh 1981)', 'Sinh': ['Vũ Lệ Quyên', '2 tháng 4, 1981', '(44\xa0tuổi)', 'Hà Nội', ',', 'Việt Nam'], 'năm thành lập': 1999, 'năm tan rã': None, 'Quốc tịch': 'Việt Nam', 'Tên khác': 'Thiên Nga Đen', 'Dân tộc': 'Kinh', 'Nghề nghiệp': 'Ca sĩ', 'Năm hoạt động': '1999\xa0– nay [ 1 ]', 'Phối ngẫu': ['Lê Đức Huy', '(cưới\xa02011\u2060–\u2060', 'ld.', '2020)'], 'Con cái': '1', 'Giải thưởng': 'Danh sách', 'Website': 'Facebook', 'Thể lo